In [57]:
import os
import argparse
import glob
import sys 
import yaml 
import glob

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages

from scipy import interpolate 
from scipy import signal

from pathlib import Path
sys.path.append(str(Path('.').absolute().parent.parent))
from util.paths import find, list_subdirs

import matplotlib as mpl
mpl.rcParams.update({'font.size':         24,
                     'axes.linewidth':    3,
                     'xtick.major.size':  5,
                     'xtick.major.width': 2,
                     'ytick.major.size':  5,
                     'ytick.major.width': 2,
                     'axes.spines.right': False,
                     'axes.spines.top':   False,
                     'font.sans-serif':  "Arial",
                     'font.family':      "sans-serif",
                    })

In [46]:
os.path.normpath('T:\\OptoPreyCapture\\051821\\PVCH12p10\\trial_{:d}\\*_BasicParams.h5'.format(trial))

'T:\\OptoPreyCapture\\051821\\PVCH12p10\\trial_3\\*_BasicParams.h5'

In [47]:
trial=3
fpath = glob.glob(os.path.normpath('T:\\OptoPreyCapture\\051821\\PVCH12p10\\trial_{:d}\\*_BasicParams.h5'.format(trial)))[0]

In [48]:
with pd.HDFStore(fpath) as store:
    data = store['df']
    metadata = store.get_storer('df').attrs.metadata

In [86]:
with PdfPages(fpath[:-3] + '_Plots.pdf') as pdf:
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(data['t'],data['Angle'], c='b')
    ax.plot(data['t'],data['Speed'], c='k')
    ax.plot(data['t'],data['Dist'], c='m')
    ax.axvline(x=metadata['TimeToApproach'], c='g')
    ax.axvline(x=metadata['CaptureT'], c='r')
    ax.set_title('Basic Params')
    ax.legend(['Angle', 'Speed', 'Dist. to Cricket', 'TimeToApproach', 'CaptureT'], bbox_to_anchor=(1.01, 1), fontsize=10)
    plt.tight_layout()
    pdf.savefig()  # saves the current figure into a pdf page
    plt.close()

    fig, ax = plt.subplots(figsize=(10,8))
    plot_min, plot_max = np.nanmin(data[['Mouse_x','Mouse_y','Cricket_x','Cricket_y']].to_numpy()), np.nanmax(data[['Mouse_x','Mouse_y','Cricket_x','Cricket_y']].to_numpy())
    ax.plot(data['Mouse_y'], data['Mouse_x'],c='k')
    ax.plot(data['Cricket_y'], data['Cricket_x'],c='r')
    ax.set_xlim([plot_min-1,plot_max+1])
    ax.set_ylim([plot_min-1,plot_max+1])
    ax.set_title('DLC Tracking')
    ax.legend(['Mouse', 'Cricket'],bbox_to_anchor=(1.01, 1), fontsize=10)
    ax.set_aspect('equal', 'box')
    plt.tight_layout()
    pdf.savefig()  # saves the current figure into a pdf page
    plt.close()

In [132]:
csv_filepath = os.path.normpath('T:/OptoPreyCapture/csv_today.csv')
csv = pd.read_csv(csv_filepath)
csv['experiment_date'] = pd.to_datetime(csv['experiment_date'],infer_datetime_format=True,format='%m%d%Y').dt.strftime('%m%d%y')
csv = csv.loc[(csv['run_preprocessing'] == True)|(csv['run_ephys_analysis'] == True)]
csv = csv[csv['experiment_outcome']=='good'].reset_index(drop=True)
# Format Pandas Dataframe to have Trial number and Stimulus condition

cols = list(csv.keys()[:-4])
cols.append('Trial')
cols.append('LaserOn')
csv2 = pd.DataFrame(columns=cols)
for ind,row in csv.iterrows():
    for n in range(1,5):
        if '*' in row['Trial_{:d}'.format(n)]:
            csv2 = csv2.append(row[:-4].append(pd.Series([n,True],index=['Trial','LaserOn'])),ignore_index=True)
        else:
            csv2 = csv2.append(row[:-4].append(pd.Series([n,False],index=['Trial','LaserOn'])),ignore_index=True)
inds, labels = csv2['Wallpaper'].factorize()

In [205]:
df_meta = pd.DataFrame(columns=list(metadata.keys()))
for ind, row in csv2.iterrows():
    fname = str(list(Path(row['drive']+':/','OptoPreyCapture',row['experiment_date'],row['animal_name'],'trial_{:d}'.format(row['Trial'])).glob('*_BasicParams.h5'))[0])
    with pd.HDFStore(fname) as store:
        data = store['df']
        metadata = store.get_storer('df').attrs.metadata
        metadata.update(csv2[['animal_name','experiment_date','Trial','LaserOn']].iloc[ind])
        df_meta = df_meta.append(metadata, ignore_index=True)

In [208]:
df_meta.groupby('LaserOn').mean()

,MovieT,CaptureT,TimeToApproach,FreqApproach,ProbInter,ProbCapture
LaserOn,,,,,,
False,35.670076,31.129924,4.823675,0.222417,0.515631,0.482224
True,37.437917,33.072083,5.925311,0.191019,0.507018,0.604196


In [209]:
df_meta.groupby(['Trial','LaserOn']).mean()

MovieT   CaptureT  TimeToApproach  FreqApproach  ProbInter  \
Trial LaserOn                                                                  
1     False    36.966667  32.869231        5.075134      0.236224   0.521546   
      True     35.922917  31.291667        6.264285      0.220721   0.450123   
2     False    45.022222  40.046296        4.188541      0.193853   0.515520   
      True     37.900000  33.856944        6.050704      0.198577   0.529478   
3     False    29.018056  24.801389        5.050230      0.236772   0.558238   
      True     38.811111  33.622222        4.281215      0.177878   0.517781   
4     False    33.550000  28.438333        4.796532      0.212951   0.456912   
      True     36.912121  33.060606        6.887162      0.171924   0.515089   

               ProbCapture  
Trial LaserOn               
1     False       0.428205  
      True        0.650000  
2     False       0.439815  
      True        0.560516  
3     False       0.531349  
      True        0.606481  
4     False       0.531667  
      True        0.616667

In [204]:
metadata

{'MovieT': 39.75,
 'CaptureT': 36.13333333333333,
 'TimeToApproach': 0.9171580000038375,
 'FreqApproach': 0.3018867924528302,
 'ProbInter': 0.4166666666666667,
 'ProbCapture': 0.2,
 'animal_name': 'CH2p10',
 'experiment_date': '062421',
 'Trial': 4,
 'LaserOn': False}

In [183]:
row = csv2[csv2['LaserOn']==True].iloc[5]
fname = str(list(Path(row['drive']+':/','OptoPreyCapture',row['experiment_date'],row['animal_name'],'trial_{:d}'.format(row['Trial'])).glob('*_BasicParams.h5'))[0])
print(fname)
with pd.HDFStore(fpath) as store:
    data = store['df']
    metadata = store.get_storer('df').attrs.metadata

T:\OptoPreyCapture\061821\PVCH13p3RT\trial_4\061821_PVCH13p3RT_laser_on_Rig2_trial_4_BasicParams.h5


In [184]:
metadata

{'MovieT': 33.766666666666666,
 'CaptureT': 26.216666666666665,
 'TimeToApproach': 2.232742000000144,
 'FreqApproach': 0.11846001974333663,
 'ProbInter': 0.5,
 'ProbCapture': 0.5}

In [156]:
df_cols = list(csv2.keys())[:2] + list(csv2.keys())[-5:] + list(metadata.keys())

In [155]:
csv2[list(csv2.keys())[:2]+list(csv2.keys())[-4:]]

,animal_name,experiment_date,computer,drive,Trial,LaserOn
0,PVCH12p10,061821,new-monster,T,1,False
1,PVCH12p10,061821,new-monster,T,2,True
2,PVCH12p10,061821,new-monster,T,3,False
3,PVCH12p10,061821,new-monster,T,4,True
4,PVCH12p11a,061821,new-monster,T,1,False
...,...,...,...,...,...,...
79,CH2p4,062421,new-monster,T,4,False
80,CH2p10,062421,new-monster,T,1,True
81,CH2p10,062421,new-monster,T,2,False
82,CH2p10,062421,new-monster,T,3,True
